In [2]:
# Import Needed Libraries
import numpy as np
import cvxpy as cp

In [3]:
# Function
# Initial feasible trajectory (for initialization of sample safe set) using brute-force or open-loop control
def initial_feasible_trajectory():
    x = xS.copy()
    traj_x = [x.copy()]
    traj_u = []
    for _ in range(30):
        # Simple damped control toward origin
        u = np.clip(-x[0] - x[1], u_min[0], u_max[0])
        u = np.array([u]) # Reshape u to be a 1-dimensional array
        x = A @ x + B @ u
        traj_x.append(x.copy())
        traj_u.append(u)
        if np.linalg.norm(x - xF) < tol:
            break
    return np.array(traj_x), np.array(traj_u)

In [4]:
# Function
# LMPC optimization solver with sampled safe set and terminal cost
# Use in each iteration of LMPC
def solve_lmpc(A, B, Q, R, N, x0, ss_x, ss_cost, x_min, x_max, u_min, u_max):
    n, m = B.shape
    x = cp.Variable((n, N+1))
    u = cp.Variable((m, N))
    cost = 0
    constraints = [x[:, 0] == x0]

    # Construct cost and constraints
    for k in range(N):
        cost += cp.quad_form(x[:, k], Q) + cp.quad_form(u[:, k], R)
        constraints += [
            x[:, k+1] == A @ x[:, k] + B @ u[:, k],
            x_min <= x[:, k], x[:, k] <= x_max,
            u_min <= u[:, k], u[:, k] <= u_max
        ]

    # Terminal constraint: final state must match a sampled safe point
    if len(ss_x) > 0:
        # Terminal cost-to-go from sampled safe set
        # Find the closest point in the safe set
        min_distance = float('inf')  # Initialize with a large value
        closest_point_index = -1

        for i, s in enumerate(ss_x):
            distance = cp.norm(x[:, N] - s, 2)
            # Use cp.min to find the minimum distance within cvxpy
            if distance.value is not None and distance.value < min_distance:  # Compare distance values
                min_distance = distance.value
                closest_point_index = i

      # Add the terminal cost associated with the closest point
        if closest_point_index != -1:
          cost += ss_cost[closest_point_index]


    # Terminal state should still satisfy bounds
    constraints += [x_min <= x[:, N], x[:, N] <= x_max]

    prob = cp.Problem(cp.Minimize(cost), constraints) # problem
    prob.solve(solver=cp.OSQP) # apply solver to problem

    return u[:, 0].value

In [5]:
# Problem setup

# Dynamics
A = np.array([[1, 1],
              [0, 1]])
B = np.array([[0],
              [1]])

# Cost Weights
Q = np.eye(2)
R = np.eye(1)

# Constraints
x_min = np.array([-4, -4])
x_max = np.array([4, 4])
u_min = np.array([-1])
u_max = np.array([1])

# Horizon
N = 4

# Inital/Final States
xS = np.array([-3.95, -0.05]) # inital state x0
xF = np.array([0.0, 0.0]) # final desired state [0,0]
tol = 1e-3
max_iters = 3
max_steps = 100

In [6]:
# Implement

# LMPC loop initalizations
safe_set = []
safe_cost = []
trajectories = []
print("Starting LMPC...")

# Initialize with a feasible trajectory for first iteration
x_traj, u_traj = initial_feasible_trajectory()
trajectories.append((x_traj, u_traj))
safe_set.extend(x_traj)
safe_cost.extend([np.sum([x.T @ Q @ x + u.T @ R @ u for x, u in zip(x_traj[t:], u_traj[t:] + [np.zeros_like(u_traj[0])])]) for t in range(len(x_traj))])

# Iterate LMPC loop
for j in range(1, max_iters):
    print(f"\n=== Iteration {j} ===")
    x = xS.copy()
    x_traj = [x.copy()]
    u_traj = []
    for step in range(max_steps):
        u = solve_lmpc(A, B, Q, R, N, x, safe_set, safe_cost, x_min, x_max, u_min, u_max)
        x = A @ x + B @ u
        x_traj.append(x.copy())
        u_traj.append(u)
        if np.linalg.norm(x - xF) < tol:
            break

    # Store trajectory
    x_traj = np.array(x_traj)
    u_traj = np.array(u_traj)
    trajectories.append((x_traj, u_traj))


    # Update safe set and cost
    safe_set.extend(x_traj)

    for t in range(len(x_traj)):
      sub_cost = sum(
          x.T @ Q @ x + u.T @ R @ u
          for x, u in zip(x_traj[t:-1], u_traj[t:])
      )
      sub_cost += x_traj[-1].T @ Q @ x_traj[-1]  # terminal state cost
      safe_cost.append(sub_cost)

    # Compute and print iteration cost
    iter_cost = sum(
    x.T @ Q @ x + u.T @ R @ u
    for x, u in zip(x_traj[:-1], u_traj)
)
    final_cost = x_traj[-1].T @ Q @ x_traj[-1]
    total_cost = iter_cost + final_cost
    print(f"Iteration {j} cost: {total_cost:.4f}")

    print(f"Sampled Safe Set size: {len(safe_set)} points")

    # Debug: print first few states and controls of the current trajectory
    print("Initial state:", x_traj[0])
    print("First few controls:", u_traj[:3].flatten())
    print("First few states after control:", x_traj[1:4])

    print(f"Last point in new trajectory: {x_traj[-1]}")
    print(f"Distance to final target xF: {np.linalg.norm(x_traj[-1] - xF):.4f}")


print("\nLMPC completed.")



Starting LMPC...

=== Iteration 1 ===
Iteration 1 cost: 49.9291
Sampled Safe Set size: 44 points
Initial state: [-3.95 -0.05]
First few controls: [ 0.99999973  0.46000032 -0.47199996]
First few states after control: [[-4.          0.94999973]
 [-3.05000027  1.41000005]
 [-1.64000022  0.9380001 ]]
Last point in new trajectory: [ 0.00074408 -0.0006309 ]
Distance to final target xF: 0.0010

=== Iteration 2 ===
Iteration 2 cost: 49.9291
Sampled Safe Set size: 57 points
Initial state: [-3.95 -0.05]
First few controls: [ 0.99999973  0.46000032 -0.47199996]
First few states after control: [[-4.          0.94999973]
 [-3.05000027  1.41000005]
 [-1.64000022  0.9380001 ]]
Last point in new trajectory: [ 0.00074408 -0.0006309 ]
Distance to final target xF: 0.0010

LMPC completed.
